In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import sys
sys.path.append('../')

In [17]:
import os
from src.pcai_datamodule import PatchDataModule
from src.pcai_litmodule import LitModuleClasAdversarial

In [18]:
dataset_dir = '../data/tma_dataset/'
image_base_dir = os.path.join(dataset_dir, "images")
mask_base_dir = os.path.join(dataset_dir, "masks/masks/")
metadata_path = os.path.join(dataset_dir, 'metadata/metadata.csv')

In [19]:
datamodule = PatchDataModule(
    metadata_path=metadata_path,
    image_base_dir=image_base_dir,
    mask_base_dir=mask_base_dir,  
    label_cols_clas=['event_duration', 'event_indicator'],
    label_col_domain=['domain'],
    patch_size=14,
    n_patches=5, # return 5 patches per bag
)

datamodule.setup("fit")

In [20]:
sample = next(iter(datamodule.train_dataloader()))
len(sample)

4

In [21]:
sample[0].shape

torch.Size([2, 5, 3, 14, 14])

In [22]:
from src.callbacks.gradient_reversal_alpha_updater import GradientReversalAlphaUpdater

In [23]:
alpha_updater = GradientReversalAlphaUpdater(max_epochs=3)

In [24]:
litmodule = LitModuleClasAdversarial()

In [25]:
from pytorch_lightning import Trainer

In [26]:
trainer = Trainer(accelerator="cpu", max_epochs=5, callbacks=[alpha_updater])

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [27]:
trainer.fit(model=litmodule, datamodule=datamodule)


  | Name           | Type             | Params
----------------------------------------------------
0 | net            | AdversarialNet   | 6.5 M 
1 | criterion_clas | CrossEntropyLoss | 0     
2 | criterion_adv  | CrossEntropyLoss | 0     
3 | metrics        | ModuleDict       | 0     
----------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
25.913    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/pyth

Training: 0it [00:00, ?it/s]

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


...

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless.

Validation: 0it [00:00, ?it/s]

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless.

Validation: 0it [00:00, ?it/s]

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless.

In [28]:
predictions = trainer.predict(model=litmodule, datamodule=datamodule)
predictions

/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 1it [00:00, ?it/s]

[{'x_hat_clas': tensor([[0.0037, 0.2245]]),
  'x_hat_adv': tensor([[0.0002, 0.0111, 0.0698]]),
  'y_clas': tensor([0]),
  'y_adv': tensor([0]),
  'patient_id': tensor([0]),
  'event_duration': tensor([100]),
  'event_indicator': tensor([0]),
  'domain': ['A'],
  'sample_id': tensor([0]),
  'filepath': ['0.tif'],
  'split': ['train'],
  'patch_info': tensor([[[0, 3],
           [1, 1],
           [1, 2],
           [1, 3],
           [1, 4],
           [1, 5],
           [2, 0],
           [2, 1],
           [2, 2],
           [2, 3],
           [2, 4],
           [2, 5],
           [2, 6],
           [3, 0],
           [3, 1],
           [3, 2],
           [3, 3],
           [3, 4],
           [3, 5],
           [3, 6],
           [4, 0],
           [4, 1],
           [4, 2],
           [4, 3],
           [4, 4],
           [4, 5],
           [4, 6],
           [5, 1],
           [5, 2],
           [5, 3],
           [5, 4],
           [5, 5],
           [6, 2],
           [6, 3],
    